### 7. Variational Inference

设目标函数是$f(\theta)$（$\theta \in \mathbf{R}^{n}$），若$\theta$各分量之间存在复杂的依赖关系，使得$f(\theta)$很难计算。

VI的目标是寻找一个简单的$q(z)$，用$q(z)$去逼近$f(\theta)$.

<img src='attachment/69. 贝叶斯模型的学习.png' style='zoom:50%'/>

#### 平均场（Mean Field）

假设$q(z)$对$~z~$的所有分量都是独立的，即满足

$$
q(z) = q(z_{1})q(z_{2})\cdots q(z_{n}) \tag{1}
$$

此时的变分分布称为平均场（mean field）。

#### KL Divergence

$$
\begin{eqnarray}
&\quad& KL\left(q(\theta) \mid p(\theta|X)\right) \\
\quad \\
&=& E_{q}\left[\log \frac{q(\theta)}{p(\theta|X)} \right] \\
\quad \\
&=& E_{q}\left[\log q(\theta) - \log p(\theta|X) \right] \\
\quad \\
&=& E_{q}\left[\log q(\theta) - \log p(\theta, X) + \log P(X) \right] \\
\quad \\
&=& -\underset{ELBO}{\underbrace{\left( E_{q}\left[\log p(\theta, X) \right] - E_{q}\left[\log q(\theta) \right] \right)} } + Constant \tag{2}
\end{eqnarray}
$$

#### LDA模型的VI

<img src='attachment/69. VI.png' style='zoom:50%'/>

目标是估算后验概率 $P(\theta, \phi, Z| \alpha, \beta, w)$

设基于平均场的变分分布为 $q(\theta, \phi, Z)$，有

$$
q(\theta, \phi, Z) = q(\theta|\gamma)~q(\phi|\rho)~q(Z|\varepsilon) \tag{3}
$$

其中，$\gamma, \rho$是Dirichlet分布参数，$\varepsilon$是多项式分布参数。

则ELBO为:  

$$
ELBO = \underset{①}{E_{q}\left[ \log P(w, \theta, \phi, Z) \right]} - \underset{②}{E_{q}\left[\log q(\theta, \phi, Z) \right]} \tag{4}
$$

对ELBO的第1项

$$
\begin{eqnarray}
&\quad& E_{q}\left[ \log P(w, \theta, \phi, Z) \right]\\
&=& E_{q}\left[ \log P(\theta|\alpha) ~ P(Z|\theta) ~ P(\phi|\beta) ~ P(w|Z,\phi) \right] \\
\quad \\
&=& E_{q}\left[ \log P(\theta|\alpha)  \right] + E_{q}\left[\log P(Z|\theta) \right] + E_{q}\left[\log P(\phi|\beta)\right] + E_{q}\left[\log P(w|Z,\phi) \right]   \tag{5}
\end{eqnarray}
$$

对ELBO的第2项

$$
\begin{eqnarray}
&\quad& E_{q}\left[\log q(\theta, \phi, Z) \right] \\
&=& E_{q}\left[\log q(\theta|\gamma) \right] + E_{q}\left[\log q(\phi|\rho) \right] + E_{q}\left[\log q(Z|\varepsilon) \right] \tag{6}
\end{eqnarray}
$$

因此，Maximize ELBO为

$$
\text{maximize} \ E_{q}\left[ \log P(\log \theta|\alpha)  \right] + E_{q}\left[\log P(Z|\theta) \right] + E_{q}\left[\log P(\phi|\beta)\right] + E_{q}\left[\log P(w|Z,\phi) \right] \\
- E_{q}\left[\log q(\theta|\gamma) \right] - E_{q}\left[\log q(\phi|\rho) \right] - E_{q}\left[\log q(Z|\varepsilon) \right] \tag{7}
$$

以第二项计算为例: 

$$
\begin{eqnarray}
&\quad& E_{q}\left[\log P(Z|\theta) \right] \\
&=& E_{q}\left[\log \prod_{i=1}^{N} \prod_{j=1}^{N_{i}} P(Z_{ij}|\theta_{i}) \right] \\
&=& E_{q}\left[\sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \log  P(Z_{ij}|\theta_{i}) \right] \\
&=& E_{q}\left[\sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \log  \prod_{k=1}^{K} \theta_{ik}^{I(Z_{ij} ~=k )} \right] \\
&=& E_{q}\left[\sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K}I(Z_{ij} =k ) \log \theta_{ik} \right] \\
&=& \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} E_{q}\left[I(Z_{ij} =k ) \right] \cdot E_{q}\left[\log \theta_{ik} \right] \\
&=& \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} \color{red}{\varepsilon_{ijk}} \cdot \left[\psi(\gamma_{ik}) - \psi\left(\sum_{s=1}^{K}\gamma_{is} \right) \right]
\end{eqnarray}
$$

同理，计算最后一项

$$
\begin{eqnarray}
&\quad& E_{q}\left[\log q(Z|\varepsilon) \right] \\
&=& E_{q}\left[\log \prod_{i=1}^{N} \prod_{j=1}^{N_{i}} q(Z_{ij}|\varepsilon_{ij}) \right] \\
&=& E_{q}\left[\log \prod_{i=1}^{N} \prod_{j=1}^{N_{i}} \prod_{k=1}^{K} \varepsilon_{ijk}^{I(Z_{ij}~=k)} \right] \\
&=& E_{q}\left[\sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} I(Z_{ij}~=k) \log \varepsilon_{ijk} \right] \\
&=& \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} E_{q} \left[I(Z_{ij}~=k) \right] \cdot E_{q} \left[\log \varepsilon_{ijk} \right] \\
&=&  \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} \varepsilon_{ijk} \cdot \log \varepsilon_{ijk}
\end{eqnarray}
$$

最终得到ELBO的表达式为: 

$$
\begin{eqnarray}
f(\varepsilon, \gamma, \rho) &=& \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} \color{red}{\varepsilon_{ijk}} \cdot \left[\psi(\gamma_{ik}) - \psi\left(\sum_{s=1}^{K}\gamma_{is} \right) \right]  \\
\quad \\
&\quad& + \sum_{k=1}^{K} \left[\log \Gamma \left(\sum_{s=1}^{K}\alpha_{s} \right) - \sum_{s=1}^{K} \log \Gamma(\alpha_{s}) \right] + \sum_{k=1}^{K}(\alpha_{k}-1) \left[\psi(\gamma_{ik}) - \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) \right]\\ 
\quad \\
&\quad& + \sum_{k=1}^{K} \left[\log \Gamma \left(\sum_{v=1}^{|V|}\beta_{v} \right) - \sum_{v=1}^{|V|} \log \Gamma(\beta_{v}) \right] + \sum_{v=1}^{|V|}(\beta_{v}-1) \left[\psi(\rho_{kv}) - \psi \left(\sum_{t=1}^{|V|}\rho_{kt} \right) \right] \\
\quad \\
&\quad& + \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} \sum_{v=1}^{|V|} \varepsilon_{ijk} \cdot w_{ij}^{v}\left[ \psi(\rho_{kv}) - \psi \left(\sum_{t=1}^{|V|}\rho_{kt} \right) \right] \\
\quad \\
&\quad& - \sum_{i=1}^{N} \left[ \log \Gamma \left( \sum_{s=1}^{K}\gamma_{is} \right) - \sum_{k=1}^{K} \log \Gamma(\gamma_{ik}) \right] -  \sum_{k=1}^{K}(\gamma_{ik}-1) \left[\psi(\gamma_{ik}) - \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
\quad \\
&\quad& - \sum_{i=1}^{N} \sum_{j=1}^{N_{i}} \sum_{k=1}^{K} \varepsilon_{ijk}\cdot \log \varepsilon_{ijk} \\
\quad \\
&\quad& - \sum_{K=1}^{k} \left[ \log \Gamma \left( \sum_{V=1}^{|V|}\rho_{kv} \right) - \sum_{v=1}^{|V|} \log \Gamma(\rho_{kv})\right ] - \sum_{v=1}^{|V|}(\rho_{kv}-1) \left[\psi(\rho_{kv}) - \psi \left(\sum_{t=1}^{|V|}\rho_{kt
} \right) \right] \tag{8}
\end{eqnarray}
$$

使用坐标上升法Maximize $f(\varepsilon, \gamma, \rho)$:

$\varepsilon^{*} \leftarrow Maximize ~ f(\varepsilon, \gamma^{*}, \rho^{*})$

$\gamma^{*} \leftarrow Maximize ~ f(\varepsilon^{*}, \gamma, \rho^{*})$

$\rho^{*} \leftarrow Maximize ~ f(\varepsilon^{*}, \gamma^{*}, \rho)$

以计算$~\gamma_{ik}~$为例：

$$
\begin{align}
f(\gamma_{ik}) &= \sum_{j=1}^{N_{i}} \varepsilon_{ijk} \left[\psi(\gamma_{ik}) - \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
&+ (\alpha_{k}-1)\left[\psi(\gamma_{ik}) - \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
&- \log \Gamma \left(\sum_{s=1}^{K}\gamma_{is} \right) + \log \Gamma (\gamma_{ik}) \\
&- (\gamma_{ik}-1) \left[\psi(\gamma_{ik}) - \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) \right]
\end{align}
$$ 

令 $f(\gamma_{ik})$对$~\gamma_{ik}~$的导数为零，解$~\gamma_{ik}~$:


$$
\begin{align}
\frac{\partial f(\gamma_{ik})}{\partial \gamma_{ik}} &= \sum_{j=1}^{N_{i}} \varepsilon_{ijk} \left[\psi^{\prime}(\gamma_{ik}) - \psi^{\prime} \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
&+ (\alpha_{k}-1)\left[\psi^{\prime}(\gamma_{ik}) - \psi^{\prime} \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
&-  \psi \left(\sum_{s=1}^{K}\gamma_{is} \right) +  \psi (\gamma_{ik}) \\
&- (\gamma_{ik}-1) \left[\psi^{\prime}(\gamma_{ik}) - \psi^{\prime} \left(\sum_{s=1}^{K}\gamma_{is} \right) \right] \\
&- \psi (\gamma_{ik}) + \psi \left(\sum_{s=1}^{K}\gamma_{is} \right)  =0  \\
\end{align}
$$ 

$$
\begin{eqnarray}
&\Rightarrow& \ \left(\sum_{j=1}^{N_{i}}\varepsilon_{ijk} + \alpha_{k} -1 - \gamma_{ik} +1 \right)\underset{\neq 0}{\left(\psi^{\prime} (\gamma_{ik}) - \psi^{\prime} \left( \sum_{s=1}^{K}\gamma_{is} \right)\right)} = 0 \\
&\Rightarrow& \ \left(\sum_{j=1}^{N_{i}}\varepsilon_{ijk} + \alpha_{k} - \gamma_{ik}  \right) = 0 \\
\quad \\
&\therefore& \gamma_{ik} = \sum_{j=1}^{N_{i}}\varepsilon_{ijk} + \alpha_{k} \tag{9}
\end{eqnarray}
$$

同理： 

$$
\begin{eqnarray}
\varepsilon_{ijk} &=& \frac{\varepsilon_{ijk}}{\sum_{s=1}^{K}\varepsilon_{ijs}  } \\
\rho_{kv} &=& \rho_{v} + \sum_{i=1}^{N}\sum_{j=1}^{N_{i}}\varepsilon_{ijk} I(w_{ij}=v)
\end{eqnarray}
$$